In [1]:
import sys
sys.path.append('/usr/lib/python3/dist-packages')
import os
import pandas as pd
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
from django.conf import settings
os.environ["DJANGO_SETTINGS_MODULE"] = "mysite.settings"
import django
django.setup()
from tqdm import tqdm, tqdm_notebook
import glob
import numpy as np
import requests
import pytz
import time
import glob
import itertools

/home/tserekh/env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
from main_map.models import OrganizationNatClass, House

In [3]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

In [4]:
def get_words_dates(path):
    filename= '.'.join(path.split('/')[-1].split('.')[:-1])
    date = filename[-17:]
    date = datetime.datetime.strptime(date, '%Y-%m-%d_%H%M%S')
    word = filename[:-17]
    return (word, date,path)

In [5]:
from pyproj import Proj, transform

PROJ3857  = Proj("+init=EPSG:3857")
PROJ4326 = Proj("+init=EPSG:4326")

In [6]:
comfig_file_path = '../main_config.txt'




f = open(comfig_file_path)
conf_dic = json.loads(f.read())
f.close()
data_path = conf_dic['abc_data_path']
yandex_keys = conf_dic['yandex_org_search_keys']

main_name = 'yandex/'
sub_name = 'raw_org_search/'
sub_name_json = 'raw_org_search_json/'

if not os.path.exists(data_path + main_name):
    os.mkdir(data_path + main_name)
if not os.path.exists(data_path + main_name + sub_name):
    os.mkdir(data_path + main_name + sub_name)
if not os.path.exists(data_path + main_name + sub_name_json):
    os.mkdir(data_path + main_name + sub_name_json)

In [7]:
curr_api_num = 0
def resp(min_lat, min_lon, max_lat, max_lon, text, apikey, n_results):
    min_lat = str(min_lat)
    min_lon = str(min_lon)
    max_lat = str(max_lat)
    max_lon = str(max_lon)

    # text = 'банкомат'
    rspn = '1'
    results = str(n_results)
    timeout = 10
    ######################### прогоняем несколько раз если перевыпонил
    for j in range(1):
        url = 'https://search-maps.yandex.ru/v1/?' + '&lang=ru_RU' + '&text=' + text
        url += '&rspn=' + rspn + '&type=' + '&results=' + results + '&apikey=' + apikey
        url += '&bbox=' + min_lon + ',' + min_lat + '~' + max_lon + ',' + max_lat
        while True:
            try:
                response = requests.get(url, timeout=timeout)
                break
            except requests.exceptions.Timeout:
                timeout *= 2
                time.sleep(timeout)
                print('time exeption/ timeout = ',timeout)
            
        data = response.json()

        if('message' in data):
            if data['message'] == 'invalid key':
                return None
        if 'features' in data:
            N = len(data['features'])
        else:
            N = 0
        # print(lat_delta)
        # print(lon_delta)
        print(N, end=' ')

    sub_dic_list = []
    for i in range(N):
        sub_dic = {}
        try:
            sub_dic['lon'] = data['features'][i]['geometry']['coordinates'][0]
            sub_dic['lat'] = data['features'][i]['geometry']['coordinates'][1]
            sub_dic['Chain_name'] = data['features'][i]['properties']['name']
            sub_dic['id'] = data['features'][i]['properties']['CompanyMetaData']['id']
        except:
            continue

        if 'Features' in data['features'][i]['properties']['CompanyMetaData']:
            sub_dic['features'] = json.dumps(data['features'][i]['properties']['CompanyMetaData']['Features'], ensure_ascii=False)
        Categor = data['features'][i]['properties']['CompanyMetaData']['Categories']
        for categor_name in Categor:
            if 'class' in categor_name.keys():
                sub_dic['class'] = categor_name['class']
                sub_dic['name'] = categor_name['name']
        sub_dic['address'] = data['features'][i]['properties']['description']
        sub_dic_list.append(sub_dic.copy())
    df = pd.DataFrame(sub_dic_list)
    return df, data

In [8]:
sq_side = 50000

In [9]:

features =['x','y']
objects = House.objects.all().values().values_list(*features)

coord_set = set()

for object_ in objects:
    x, y = object_[0], object_[1]
    x, y = sq_side*round(x/sq_side), sq_side*round(y/sq_side)
    coord_set.add((x, y ))

In [10]:
len(coord_set)

2096

In [11]:
n_results = 5000

In [12]:
import geopy

In [13]:
38.64721,54.25524,38.64733,54.25973,38.65500,54.25966,38.65488,54.25517

(38.64721, 54.25524, 38.64733, 54.25973, 38.655, 54.25966, 38.65488, 54.25517)

In [14]:
a1,b1 = transform(PROJ4326, PROJ3857, 38.64721,54.25524)

In [15]:
a2,b2  = transform(PROJ4326, PROJ3857, 38.64733,54.25973)

In [16]:
a1 - a2

-13.358338894322515

In [17]:
b1 - b2

-855.6534411758184

In [21]:
lat, lon = 55.753215, 37.712336

x, y = transform(PROJ4326, PROJ3857, lon, lat )

new_lon, new_lat = transform(PROJ3857, PROJ4326, x+1000, y )[::-1]

from geopy.distance import vincenty

p1 = (lat, lon)
p2 = (new_lon, new_lat)
d=  vincenty(p1, p2)
d.m

564.0500864942728

In [22]:
moscow_side = 10000
#с новой москвой
#lat, lon = 55.616827, 37.286707
#x, y = transform(PROJ4326, PROJ3857, lon, lat)

# чисто москва до мкада
lat, lon = 55.762672, 37.625627
x, y = transform(PROJ4326, PROJ3857, lon, lat)

# зеленоград
#lat, lon = 55.989807, 37.195204
#x, y = transform(PROJ4326, PROJ3857, lon, lat)

#горки
#lat, lon = 55.732574, 37.002793
#x, y = transform(PROJ4326, PROJ3857, lon, lat)

##x, y = transform(PROJ4326, PROJ3857, 37.619760, 55.763635)

In [23]:
sides = [moscow_side]

In [24]:
moscow_side

10000

In [25]:
#с новой москвой
#X = x + np.arange(-5.3e5, 5.3e5,moscow_side)
#Y = y + np.arange(-5.3e5, 5.3e5,moscow_side)

# чисто москва до мкада
X = x + np.arange(-2.3e5, 2.3e5,moscow_side)
Y = y + np.arange(-2.3e5, 2.3e5,moscow_side)

# зеленоград
#X = x + np.arange(-5e4, 5e4,moscow_side)
#Y = y + np.arange(-5e4, 5e4,moscow_side)

# горки
#X = x + np.arange(-8e4, 8e4,moscow_side)
#Y = y + np.arange(-8e4, 8e4,moscow_side)

In [26]:
coord_lists = [list(itertools.product(X,Y))]

In [27]:
len(list(itertools.product(X,Y)))

2116

In [30]:
paths = glob.glob('/home/tserekh/abc_data/yandex/raw_org_search_json/*')
df_paths = pd.DataFrame(paths,columns = ['path'])

df_paths['word'] = df_paths['path'].apply(lambda path: get_words_dates(path)[0])
df_paths['date'] = df_paths['path'].apply(lambda path: get_words_dates(path)[1])
df_paths = df_paths[df_paths['date']>pd.datetime(2018,8,20,1,4)]
df_paths = df_paths.sort_values('date', ascending=False)
df_paths = df_paths.drop_duplicates('word',keep='first')

new_words = [
'Банк',
'Банкомат',
'Аптека',
'Парикмахерская',
'Ресторан',
'Супермаркет',
'Магазин продуктов',
'АЗС',
'Автоаксессуары',
'Автомойка'
]

texts = new_words
# new_words = set(new_words) - set(df_paths['word'].values)
# new_words = list(new_words)
# texts = new_words + list(df_paths['word'].values)

In [ ]:
for text in texts:
    print(text)
    df_orgs = pd.DataFrame()
    data_orgs = []
    key_ind = 0
    apikey = yandex_keys[key_ind]
    for i in (range(len(coord_lists))):
        print(i)
        coord_list = coord_lists[i]
        sq_side = sides[i]
        for coord in tqdm_notebook(coord_list):
            lon_min, lat_min = transform(PROJ3857,PROJ4326, coord[0] - sq_side/2, coord[1] - sq_side/2)
            lon_max, lat_max = transform(PROJ3857,PROJ4326, coord[0] + sq_side/2, coord[1] + sq_side/2)
            while True:
                response = resp(lat_min, lon_min, lat_max, lon_max, text, apikey, n_results)
                if response is not None:
                    df, data = response
                    break
                else:
                    key_ind+=1
                    print('switch to key',key_ind)
                    apikey = yandex_keys[key_ind]

            df_orgs = pd.concat([df_orgs,df])
            data_orgs.append(data)
        
    now = datetime.datetime.now(tz = pytz.timezone('Europe/Moscow'))
    now = datetime.datetime.strftime(now,format = '%Y-%m-%d_%H%M%S')


    df_orgs.to_csv(data_path + main_name + sub_name + text + now + '.csv',sep='@')
    f = open(data_path + main_name + sub_name_json + text + now + '.json',mode = 'w')
    f.write(json.dumps(data_orgs))
    f.close()

Банк
0


0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 

/home/tserekh/env/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 19 0 0 2 0 6 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 8 84 1 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0 1 2 16 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 2 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 1 0 0 0 7 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 9 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 7 0 0 0 2 0 0 0 0 0 0 1 5 0 0 0 0 0 1 0 3 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 9 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 2 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 23 1 10 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0 0 0 1 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 3 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 2 0 0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 82 0 0 8 0 11 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 21 2 0 0 0 0 0 0 0 0 0 0 0 0 1 1 54 187 1 1 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 28 1 0 0 0 0 0 0 0 0 0 0 0 switch to key 1
0 0 0 

In [33]:
text

'Автомойка'

In [32]:
df_orgs

,Chain_name,address,class,features,id,lat,lon,name
0,Авто Мойка,"Россия, Московская область, Можайский городско...",car wash,NaN,126747689793,55.469624,35.528042,Автомойка
0,"Автосервис, шиномонтаж, диагностика","Оборонная ул., 48, п. г. т. Уваровка, Россия",car wash,"[{""id"": ""car_brand"", ""type"": ""enum"", ""name"": ""...",38744718584,55.523787,35.601176,Автомойка
0,Автомойка,"ул. Рогова, 9, посёлок Кировский, Россия",dry cleaning,"[{""id"": ""dry_cleaners_"", ""type"": ""enum"", ""name...",40343507466,56.212966,35.659793,Химчистка
0,Профи,"28, посёлок Новолотошино, Россия",car wash,"[{""id"": ""car_wash"", ""type"": ""bool"", ""name"": ""м...",1743150263,56.243601,35.647458,Автомойка
1,Автомойка и пятнышко,"Туровский пр., 5, п. г. т. Лотошино, Россия",car wash,"[{""id"": ""car_valeting"", ""type"": ""bool"", ""name""...",1784860028,56.245190,35.655472,Автомойка
2,Автомойка,"Россия, Московская область, Лотошинский район,...",car wash,NaN,101302855524,56.250822,35.647279,Автомойка
0,Автомойка,"Россия, Московская область, Волоколамский район",car wash,NaN,169924937836,56.028172,35.746800,Автомойка
0,AutoHouse,"Россия, Тверская область, Калининский район, д...",tire fitting,"[{""id"": ""car_wash"", ""type"": ""bool"", ""name"": ""м...",1251221448,56.488420,35.776440,Шиномонтаж
0,Автомойка,"15А, Медынь, Россия",car wash,NaN,77113836767,54.963227,35.846235,Автомойка
0,Автосервис Ком Авто,"4Б, село Никольское, Россия",tire fitting,"[{""id"": ""payment_by_credit_card"", ""type"": ""boo...",1153048940,56.826585,35.823449,Шиномонтаж


In [34]:
import psycopg2

In [ ]:
"bname=test user=postgres password=secret"

In [ ]:
psycopg2.connect()

In [19]:
now = datetime.datetime.now(tz = pytz.timezone('Europe/Moscow'))
now = datetime.datetime.strftime(now,format = '%Y-%m-%d_%H%M%S')


df_orgs.to_csv(data_path + main_name + sub_name + text + now + '.csv',sep='@')
f = open(data_path + main_name + sub_name_json + text + now + '.json',mode = 'w')
f.write(json.dumps(data_orgs))
f.close()

NameError: name 'df_orgs' is not defined

In [20]:
data_orgs[0]

NameError: name 'data_orgs' is not defined